In [79]:
#'''
import numpy as np
import pandas as pd
import urllib.request
from sklearn.mixture import GaussianMixture
import math
import random
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from collections import Counter
#'''
import time

In [3]:
plt.rcParams['figure.figsize'] = (20.0, 10.0)

In [82]:
Reference = {}
wordsum = 0

In [4]:
def _isNum(str):
    if(any(dig.isdigit() for dig in str) == True):
        return True
    else:
        return False

def _parse(s):
    tuples = s.split('), ')
    out = []
    for x in tuples:
        #print(x)
        if(len(x.strip('[]()""\n').split(', ')) != 4):
            print("error string: ", s)
            raise Exception ("Error String")
        a,b,c,d = x.strip('[]()""\n').split(', ')
        a = a.replace("'", "")
        d = d.replace("'", "")
        #print(a, b, c, d)
        if(_isNum(b) == True):
            b = float(b)
        else:
            b = b.replace("'", "")
        out.append([str(a), str(b), float(c), str(d)])
    return out

In [59]:

def getListofLowCohessionWords(pin, coh):
    urlstr = "http://abel.lis.illinois.edu/cgi-bin/cohese/search.py?PMIDs="
    pmidurl = urlstr + str(pin)
    pmidurl = pmidurl.replace("]", "").replace("[", "")
    print(pmidurl)
    with urllib.request.urlopen(pmidurl) as url:
        string_data = url.read().decode("utf-8")
        list_words = _parse(string_data)
        
        try:
            #global word_list
            
            worddf = pd.DataFrame(list_words, columns=['Word', 'CohessionScore', 'Freq', 'Association'])
            # convert stopwords into cohession score = -1
            #print("step 1")
            worddf.loc[worddf['CohessionScore'] == 'stopword', 'CohessionScore'] = 0
            worddf.loc[worddf['CohessionScore'] == None, 'CohessionScore'] = 0
            worddf.loc[worddf['CohessionScore'] == "None", 'CohessionScore'] = 0
            #print("step 2")
            # remove any remaining spaces
            worddf.Association = worddf.Association.str.strip()
            #print("step 3")
            worddf.Word = worddf.Word.str.strip()
            #print("step 4")
            # convert CohessionScore from object to numeric
            worddf.CohessionScore = pd.to_numeric(worddf.CohessionScore)
            #print("step 5")
            
            worddf = worddf.loc[(worddf['CohessionScore'] < coh) & (worddf['CohessionScore'] > 0), ['Word', 'Freq']]
            
            global wordsum
            wordsum = wordsum + worddf['Freq'].sum(axis = 0)
            
            word_dict = worddf.set_index('Word')['Freq'].to_dict()
            return word_dict
            
        except Exception as e:
            print("getListofWords error", e)
            raise e
            return 0

In [91]:
import pickle

ref1 = {}
ref2 = {}

with open('Dict2.pickle', 'rb') as handle:
    ref2 = pickle.load(handle)
    
with open('Dict1.pickle', 'rb') as handle:
    ref1 = pickle.load(handle)
    
Reference = ref1.copy()
Reference.update(ref2)

In [101]:
with open('main_dict.pickle', 'rb') as handle:
    Reference = pickle.load(handle)
Reference

{'characteristic': 423070.0,
 'assessed': 286682.0,
 'detection': 276658.0,
 'direction': 74493.0,
 'threshold': 82960.0,
 'measured': 564093.0,
 'or': 3262062.0,
 'varied': 100999.0,
 'shape': 65845.0,
 'not': 2729072.0,
 'integration': 40092.0,
 'directly': 172858.0,
 'sensitivity detection': 2762.0,
 'function': 822788.0,
 'a temporal': 4510.0,
 'provide a': 134214.0,
 'poor': 147224.0,
 'fit': 31380.0,
 'prior': 199245.0,
 'respective': 50619.0,
 'hypothesis': 206649.0,
 'according': 225094.0,
 'performance': 223766.0,
 'determined': 551153.0,
 'stage': 352356.0,
 'd': 322729.0,
 'proposed': 177671.0,
 'basis': 274184.0,
 'considered': 300594.0,
 'possible explanation': 9345.0,
 'rare': 171406.0,
 'complex': 446770.0,
 'expected': 125874.0,
 'aberrant': 20997.0,
 'examined': 583874.0,
 'experimental condition': 22300.0,
 'a selective': 31857.0,
 'advantage': 121423.0,
 'enhanced': 232144.0,
 'exposure': 291936.0,
 'evidence': 533097.0,
 'source': 193202.0,
 'control': 1086192.0,
 '

In [118]:
with open('main_dict.pickle', 'wb') as handle:
    pickle.dump(Reference, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [94]:
sum_freq = sum(Reference.values())
sum_freq

372733975.0

In [115]:
Reference_normalized = {k: v / sum_freq for k, v in Reference.items()}
Reference_normalized

{'characteristic': 0.001135045443603578,
 'assessed': 0.00076913299894381781,
 'detection': 0.00074223982399243321,
 'direction': 0.00019985567454643756,
 'threshold': 0.0002225716075385937,
 'measured': 0.0015133930305119086,
 'or': 0.0087517162877357776,
 'varied': 0.00027096805436102248,
 'shape': 0.00017665414053011938,
 'not': 0.0073217688299007351,
 'integration': 0.00010756196829119214,
 'directly': 0.000463757026710538,
 'sensitivity detection': 7.4101106559980214e-06,
 'function': 0.0022074403064544896,
 'a temporal': 1.2099782425253829e-05,
 'provide a': 0.00036007986661264245,
 'poor': 0.00039498411702340792,
 'fit': 8.4188730045335952e-05,
 'prior': 0.00053455014397332573,
 'respective': 0.00013580463117160167,
 'hypothesis': 0.00055441417702799961,
 'according': 0.00060389987255655996,
 'performance': 0.00060033700979364706,
 'determined': 0.0014786765816022004,
 'stage': 0.00094532836723564039,
 'd': 0.00086584272335249284,
 'proposed': 0.00047666972134751063,
 'basis': 0

In [117]:
with open('normalized_main_dict.pickle', 'wb') as handle:
    pickle.dump(Reference_normalized, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [119]:
print(min(Reference_normalized.values()))
print(max(Reference_normalized.values()))

3.48774216249e-08
0.020306386076
